In [1]:
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import l2beat_utils as ltwo
sys.path.pop()

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# # Get token classification
# # https://dune.com/queries/3852319
token_df = d.get_dune_data(query_id = 3852319,
    name = "tokens_by_type_base_opm",
    performance = 'large'#,
    # num_hours_to_rerun=0
)

token_df.sample(5)

2024-07-19 12:59:58.287 | INFO     | duneapi_utils:get_dune_data:58 - Results available at https://dune.com/queries/3852319
2024-07-19 13:00:00.566 | INFO     | duneapi_utils:get_dune_data:105 - ✨ Results saved as csv_outputs/tokens_by_type_base_opm.csv, with 37128 rows and 9 columns.


,blockchain,chain_id,factory_address,l1_token,num_swaps,symbol,token_address,token_classification,last_updated
19682,base,8453,None,None,845,0x,0x524a481f7f48ebbbed7773adcfa821880c537128,Not Proxy or Factory,2024-07-19 15:14:43.977406
29621,base,8453,None,None,139,freg,0x8da9efd8ba6d6a3f6c13b44ad69d55e0dbf2fbf9,Not Proxy or Factory,2024-07-19 15:14:43.977406
14679,base,8453,None,None,248,MUSK,0x9e170fe850d54a6667b8493ae201a777f86944c0,Not Proxy or Factory,2024-07-19 15:14:43.977406
36933,base,8453,None,None,101,BONZI,0x81a20bfea354903c5b973eec74c11b39a382209d,Not Proxy or Factory,2024-07-19 15:14:43.977406
33733,base,8453,None,None,244,PGOLD,0xc263663820a5a02d01576c6b06344a960cc28736,Not Proxy or Factory,2024-07-19 15:14:43.977406


In [3]:
# Get Lastest L2B aoc
l2b_df = ltwo.get_daily_aoc_by_token()
l2b_df_spc = l2b_df[l2b_df['project'].isin(['optimism','base'])]
l2b_df_spc = l2b_df_spc.rename(columns={'project':'blockchain'})

l2b_df_spc = l2b_df_spc[['dt','blockchain','token_type','asset_id','address','usd_value']]
l2b_df_spc['address'] = l2b_df_spc['address'].str.lower()

l2b_df_spc.sample(5)

,dt,blockchain,token_type,asset_id,address,usd_value
862,2024-07-19,optimism,canonical,sdai-savings-dai,0x83f20f44975d03b1b09e64809b757c47f942beea,116418.42
895,2024-07-19,optimism,native,optimism:tarot-tarot,0x1f514a61bcde34f94bc39731235690ab9da737f7,6371437.31
281,2024-07-19,base,external,base:sdex-smardex,0xfd4330b0312fdeec6d4225075b82e00493ff2e3f,2505135.07
238,2024-07-19,base,canonical,zrx-0x-protocol-token,0xe41d2489571d322189246dafa5ebde1f4699f498,2418.48
837,2024-07-19,optimism,canonical,frax-frax,0x853d955acef822db058eb8505911ed77f175b99e,3.46


In [4]:
cols = ['blockchain','token_address','l1_token','token_classification','symbol']

token_if_l1_df = token_df[cols].rename(columns={'l1_token':'address','token_address':'l2_token'})
token_if_l1_df['l1_token'] = token_if_l1_df['address']
token_if_l2_df = token_df[cols].rename(columns={'token_address':'address'})
token_if_l2_df['l2_token'] = token_if_l2_df['address']

In [5]:
df_cols = ['blockchain','address','token_classification']
token_if_l1_df = token_if_l1_df[~token_if_l1_df['address'].isna()][df_cols + ['l1_token','l2_token']].drop_duplicates()
token_if_l2_df = token_if_l2_df[~token_if_l2_df['address'].isna()][df_cols + ['l1_token','l2_token']].drop_duplicates()
# token_if_l2_df.sample(5)

In [6]:
#L2 Mapping preferred
unified_df = l2b_df_spc.merge(token_if_l2_df
                              , on=['blockchain','address'], how='left')
#L1 Mapping fallback
unified_df = unified_df.merge(token_if_l1_df
                              , on=['blockchain','address'], how='left')


In [7]:
unified_df['token_classification'] = unified_df['token_classification_x'].fillna(unified_df['token_classification_y']).fillna('Unknown')
unified_df['l1_token'] = unified_df['l1_token_x'].fillna(unified_df['l1_token_y'])
unified_df['l2_token'] = unified_df['l2_token_x'].fillna(unified_df['l2_token_y'])

unified_df = unified_df.drop(columns=['token_classification_x', 'token_classification_y','l1_token_x','l1_token_y','l2_token_x','l2_token_y'])

In [8]:
unified_df['token_classification'] = unified_df.apply(
    lambda row: 'Native ETH' if row['asset_id'] == 'eth-ether' else row['token_classification'], axis=1
)
# Override due to bad Base events
unified_df['token_classification'] = unified_df.apply(
    lambda row: 'Proxy' if row['asset_id'] == 'base:usdc-usd-coin' else row['token_classification'], axis=1
)
# Override OP predeploy
unified_df['token_classification'] = unified_df.apply(
    lambda row: 'Predeploy' if row['asset_id'] == 'op-optimism' else row['token_classification'], axis=1
)

In [9]:
# Define the priority order for token_classification
priority = {'Proxy': 1, 'Not Proxy or Factory': 2, 'Immutable Factory Created': 3}

# Add a priority column to the DataFrame based on the token_classification
unified_df['priority'] = unified_df['token_classification'].map(priority)

# Sort the DataFrame by the priority column
unified_df = unified_df.sort_values(by='priority')

# Drop duplicates based on the unique columns, keeping the first occurrence (which has the highest priority)
unique_columns = ['dt', 'blockchain', 'token_type', 'asset_id', 'address']
unified_df = unified_df.drop_duplicates(subset=unique_columns, keep='first')

# Drop the priority column as it's no longer needed
unified_df = unified_df.drop(columns=['priority'])

In [11]:
unified_df.sample(5)

,dt,blockchain,token_type,asset_id,address,usd_value,token_classification,l1_token,l2_token
0,2024-07-19,base,canonical,eth-ether,native,1.792416e+09,Native ETH,NaN,NaN
183,2024-07-19,optimism,native,optimism:fpis-frax-price-index-share,0x8368dca5ce2a4db530c0f6e535d90b6826428dee,1.619252e+04,Unknown,NaN,NaN
10,2024-07-19,base,canonical,xcn-chain,0xa2cd3d43c775978a96bdbf12d733d5a1ed94fb18,5.407342e+05,Immutable Factory Created,0xa2cd3d43c775978a96bdbf12d733d5a1ed94fb18,0x9c632e6aaa3ea73f91554f8a3cb2ed2f29605e0c
44,2024-07-19,base,canonical,pstake-pstake-finance,0xfb5c6815ca3ac72ce9f5006869ae67f18bf77006,1.315117e+05,Immutable Factory Created,0xfb5c6815ca3ac72ce9f5006869ae67f18bf77006,0x38815a4455921667d673b4cb3d48f0383ee93400
184,2024-07-19,optimism,native,optimism:fxs-frax-share,0x67ccea5bb16181e7b4109c9c2143c24a1c2205be,4.906724e+05,Not Proxy or Factory,NaN,0x67ccea5bb16181e7b4109c9c2143c24a1c2205be


In [12]:
unified_df.to_csv('csv_outputs/aoc_by_chain_token_classification.csv')

In [13]:
unified_df[unified_df['address']=='0x833589fcd6edb6e08f4c7c32d4f71b54bda02913']#.sample(5)

,dt,blockchain,token_type,asset_id,address,usd_value,token_classification,l1_token,l2_token
89,2024-07-19,base,native,base:usdc-usd-coin,0x833589fcd6edb6e08f4c7c32d4f71b54bda02913,2.943629e+09,Proxy,0x833589fcd6edb6e08f4c7c32d4f71b54bda02913,0x833589fcd6edb6e08f4c7c32d4f71b54bda02913
